# Helper: Sum Confusion Matrices
Select an arbitrary set of rows in a scorecard, and sum the confusion matrices to generate a single "total" confusion matrix. This allows for direct calculation of statistics across the full set, rather than merely averaging statistics derived from the various individual entries.  

Date: 3 March 2020  
Author: Peter Kerins  

In [ ]:
import pandas as pd
import numpy as np

#### Load and inspect scorecard data

In [ ]:
scorecard_path = "C:/Users/Peter.Kerins/World Resources Institute/Urban Land Use - Documents/WRI Results/phase_iv/scorecards_analysis/single-sheet_composite_validation.csv"
df = pd.read_csv(scorecard_path,sep=',')

In [ ]:
df.columns

In [ ]:
df.confusion

#### Separate 3-category and 6-category entries

In [ ]:
df6 = df[df.notes.str.contains('full') & df.notes.str.contains('2019')]
df3 = df[df.notes.str#### Confusion matrix scoring method.contains('reduced') & df.notes.str.contains('2019')]
print (len(df6), len(df3))

#### Define confusion matrix scoring method
Copied from `util_scoring.py` for simplicity (this notebook can be executed with just the scorecard file, not needing any other project code; helpful when VM is not currently constituted)

In [ ]:
# from util_scoring.py
def calc_confusion_details(confusion):
    n_categories = confusion.shape[0]
    # out of samples in category, how many assigned to that category
    # (true positives) / (true positives + false negatives)
    # (correct) / (samples from category)
    recalls = np.zeros(n_categories, dtype='float32')
    # out of samples assigned to category, how many belong to that category
    # (true positives) / (true positives + false positives)
    # (correct) / (samples assigned to category)
    precisions = np.zeros(n_categories, dtype='float32')

    for j in range(n_categories):
        ascribed = np.sum(confusion[:,j])
        actual = np.sum(confusion[j,:])
        correct = confusion[j,j]
        if actual:
            recalls[j] = float(correct)/float(actual)
        else:
            recalls[j] = 1e8
        if ascribed:
            precisions[j] = float(correct)/float(ascribed)
        else:
            precisions[j] = 1e8
    # what percentage of total samples were assigned to the correct category
    accuracy = confusion.trace()/float(confusion.sum())

    return recalls, precisions, accuracy

### 6-category scoring

In [ ]:
df6.confusion

In [ ]:
result = df6.confusion.apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [ ]:
np_sum = (result.sum())

In [ ]:
confusion = np_sum.reshape((6,6)).astype('uint')

In [ ]:
confusion

In [ ]:
recalls, precisions, accuracy = calc_confusion_details(confusion)

In [ ]:
print(recalls)
print(precisions)
print(accuracy)

In [ ]:
# Calculate f-score
beta = 2
f_scores = (beta**2 + 1) * precisions * recalls / ( (beta**2 * precisions) + recalls )
f_score_average = np.mean(f_scores)
print (f_scores)
print (f_score_average)

### 3-category scoring

In [ ]:
df3.confusion

In [ ]:
result = df3.confusion.apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [ ]:
np_sum = (result.sum())

In [ ]:
confusion = np_sum.reshape((3,3)).astype('uint')

In [ ]:
confusion

In [ ]:
recalls, precisions, accuracy = calc_confusion_details(confusion)

In [ ]:
print(recalls)
print(precisions)
print(accuracy)

In [ ]:
# Calculate f-score
beta = 2
f_scores = (beta**2 + 1) * precisions * recalls / ( (beta**2 * precisions) + recalls )
f_score_average = np.mean(f_scores)
print (f_scores)
print (f_score_average)